# 🎬 Video Trans Studio - AI 视频配音与口型同步
### 核心能力：
- **ASR**: Faster-Whisper (Large-v3)
- **Translation**: NLLB-200 (HuggingFace Local)
- **TTS**: Index-TTS2 (Zero-shot Voice Cloning)
- **LipSync**: Wav2Lip-GAN

> **运行环境建议**：Google Colab Tesla T4 (16GB VRAM)

In [1]:
# @title 🔍 0. 环境检查 (排查问题专用)
import sys
import os
import torch
import platform
import psutil
from IPython.display import HTML, display

def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor: return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

print("="*50)
print("💻 系统硬件环境")
print("="*50)
print(f"操作系统: {platform.system()} {platform.release()}")
print(f"处理器: {platform.processor()}")
print(f"物理核心数: {psutil.cpu_count(logical=False)}")
print(f"逻辑核心数: {psutil.cpu_count(logical=True)}")
print(f"总内存: {get_size(psutil.virtual_memory().total)}")

print("\n" + "="*50)
print("🚀 显卡 (GPU) 环境")
print("="*50)
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        prop = torch.cuda.get_device_properties(i)
        print(f"GPU {i}: {prop.name}")
        print(f"  - 显存总量: {prop.total_memory / 1024**3:.2f} GB")
        print(f"  - 计算能力: {prop.major}.{prop.minor}")
    print(f"CUDA 版本: {torch.version.cuda}")
    print(f"cuDNN 版本: {torch.backends.cudnn.version()}")
else:
    print("❌ 未检测到可用 GPU，请在 '修改' -> '笔记本设置' 中开启 T4 GPU 加速。")

print("\n" + "="*50)
print("🐍 软件开发环境")
print("="*50)
print(f"Python 版本: {sys.version}")
print(f"PyTorch 版本: {torch.__version__}")
print(f"当前路径: {os.getcwd()}")

!nvidia-smi

💻 系统硬件环境
操作系统: Linux 6.6.105+
处理器: x86_64
物理核心数: 1
逻辑核心数: 2
总内存: 12.67GB

🚀 显卡 (GPU) 环境
GPU 0: Tesla T4
  - 显存总量: 14.74 GB
  - 计算能力: 7.5
CUDA 版本: 12.6
cuDNN 版本: 91002

🐍 软件开发环境
Python 版本: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch 版本: 2.9.0+cu126
当前路径: /content
Sat Jan 31 13:11:03 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   

In [2]:
# @title 🚀 1. 环境初始化
import os
import sys
import numpy as np
from importlib.metadata import version as get_version
from packaging import version

# 1. 路径智能归位
if os.path.exists('/content/video-trans-studio'):
    os.chdir('/content/video-trans-studio')
else:
    os.chdir('/content')

# 2. 检查仓库是否存在
if not os.path.exists('core') and not os.path.exists('video-trans-studio'):
    print("📥 正在初始化仓库...")
    get_ipython().system('git clone https://github.com/infinite-gaming-studio/video-trans-studio.git')
    os.chdir('/content/video-trans-studio')

# 3. 增强版环境检测逻辑 (使用 metadata 避免内存缓存干扰)
def check_environment():
    try:
        # 检测磁盘上安装的版本，而不是内存中的版本
        t_ver = get_version("transformers")
        
        needed_dirs = ['LivePortrait', 'index-tts', 'checkpoints']
        is_dirs_ready = all(os.path.exists(d) for d in needed_dirs)
        
        if not is_dirs_ready:
            return False, "缺少核心模型目录 (LivePortrait/Index-TTS)"
            
        if version.parse(t_ver) < version.parse("4.41.0"):
            return False, f"Transformers 磁盘版本过低: {t_ver}"
            
        # 额外检查：如果内存已经加载了旧版本，提醒重启
        if 'transformers' in sys.modules:
            import transformers
            if version.parse(transformers.__version__) < version.parse("4.41.0"):
                return True, "安装已完成，但检测到旧版本缓存，请务必【重新启动会话】"

        return True, "环境就绪"
    except Exception as e:
        return False, f"检测异常: {e}"

is_ok, reason = check_environment()

if not is_ok:
    print(f"⚠️ 环境需要初始化: {reason}")
    print("🔄 正在同步代码并构建基础环境 (预计 3-5 分钟)...")
    get_ipython().system('git fetch --all && git reset --hard origin/main')
    get_ipython().system('bash setup_colab.sh')
    print("
" + "!"*50)
    print("✅ 基础环境安装成功！")
    print("⚠️ 关键一步：请点击上方菜单栏 [运行时] -> [重新启动会话] (Runtime -> Restart Session)")
    print("⚠️ 重启后，再次运行此单元格即可看到【环境就绪】。")
    print("!"*50)
elif "重新启动会话" in reason:
    print(f"⚠️ {reason}")
    print("请点击上方工具栏的 [运行时] -> [重新启动会话] ！！")
else:
    print(f"✅ {reason}！")
    import transformers
    print(f"📦 Transformers: {transformers.__version__} | NumPy: {np.__version__}")


⚠️ 环境需要初始化: Transformers 版本过低: 4.38.0
🔄 正在同步代码并构建基础环境...
Fetching origin
HEAD is now at 32c9808 fix: optimize environment detection to prevent redundant installations
🚀 Starting Video Trans Studio - MuseTalk & Index-TTS2 Edition Setup...
📦 Installing system dependencies (ffmpeg)...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
^C
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
✅ 基础环境构建完成！
⚠️ 请点击上方菜单栏：'运行时' -> '重新启动会话' (Runtime -> Restart Session)
⚠️ 重启后，再次运行此单元格即可。
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [ ]:
# @title 🔄 1.2 强制重载模块 (代码更新后必运行)
import sys
import importlib
import core.lipsync
import core.tts
import main

def reload_all():
    modules = ['main', 'core.lipsync', 'core.tts', 'core.utils', 'core.audio', 'core.asr', 'core.translator', 'config']
    for m in modules:
        if m in sys.modules:
            importlib.reload(sys.modules[m])
    print("✅ 所有核心模块已同步至仓库最新版本！")

reload_all()

In [ ]:
# @title 📂 2. 选择素材
import os
import glob
from pathlib import Path

# 核心逻辑：自动抓取服务器根目录下 sample_video 文件夹中最新上传的视频
# 在 Colab 中，根目录通常是 /content
colab_root = Path('/content')
sample_dir = colab_root / 'sample_video'

# 兼容性处理：如果不在 Colab 环境，尝试查找当前目录下的 sample_video
if not sample_dir.exists():
    sample_dir = Path('sample_video')

video_path = None

if sample_dir.exists():
    # 支持的视频格式
    valid_extensions = ['*.mp4', '*.mov', '*.avi', '*.mkv', '*.webm']
    video_files = []
    for ext in valid_extensions:
        video_files.extend(glob.glob(str(sample_dir / ext)))
    
    if video_files:
        # 按修改时间（mtime）排序，取最新的一个
        video_path = max(video_files, key=os.path.getmtime)
        print(f"✅ 成功定位服务器根目录素材: {video_path}")
        print(f"🕒 最新上传时间: {os.path.getmtime(video_path)}")
    else:
        print(f"❌ 在 {sample_dir} 文件夹中未找到任何视频文件。")
else:
    print(f"❌ 未找到目录: {sample_dir.absolute()}")
    print("💡 请确保在 Colab 左侧侧边栏的根目录（/content）下创建了 sample_video 文件夹。")

### 🛠️ 1.5 故障排除：修复模型下载失败
如果你在日志中看到 `LivePortrait failed` 或者发现模型文件大小异常（例如只有 15 字节），请运行下方单元格清理损坏的文件并重新下载。

In [ ]:
# # @title 🧹 清理并重置 LivePortrait 模型
# import os
# import shutil

# model_path = 'LivePortrait/models'
# if os.path.exists(model_path):
#     print(f"正在清理损坏的模型目录: {model_path}...")
#     shutil.rmtree(model_path)
#     print("✅ 清理完成！")
# else:
#     print("ℹ️ 未检测到模型目录，无需清理。")

# print("💡 现在你可以重新运行 '第 3 步'，程序将自动通过 HuggingFace 官方接口重新下载完整模型。")

In [ ]:
# @title ⚙️ 3. 运行全自动流水线
target_language = "en" # @param ["zh-cn", "en", "es", "fr", "ja"]
use_local_translation = True # @param {type:"boolean"}

import sys
import torch
import importlib
import os

# 确保在项目目录中运行
if os.getcwd() != '/content/video-trans-studio':
    %cd /content/video-trans-studio

# 强制重载自定义模块，防止代码缓存 (First Principles: Memory vs Disk Sync)
# 这一步至关重要，因为 Notebook 会缓存 import 的模块
modules_to_reload = ['main', 'config', 'core.tts', 'core.lipsync', 'core.utils', 'core.asr', 'core.audio', 'core.translator']
for module in modules_to_reload:
    if module in sys.modules:
        del sys.modules[module]

# 检查核心依赖，如果报错则提供修复方案
try:
    from transformers import pipeline
    from main import run_pipeline
except ImportError as e:
    print(f"❌ 模块加载失败: {e}")
    print("\n🔄 尝试自动紧急修复环境...")
    !pip install transformers==4.46.0 --force-reinstall
    print("⚠️ 环境已重置，请务必点击上方 '运行时' -> '重新启动会话'，然后再次运行此单元格。")
    sys.exit()

# 强制清理显存并执行
torch.cuda.empty_cache()

if 'video_path' in locals() and video_path:
    # 使用 print 实时刷新输出
    print(f"🎬 开始处理视频: {video_path}")
    # 顶层 await 调用异步函数
    await run_pipeline(video_path, target_lang=target_language)
else:
    print("❌ 错误：未定义 video_path，请先成功运行 '第 2 步'。")

print("\n✨ 处理全流程结束！")

In [ ]:
# @title 📺 4. 预览结果
import os
import glob
from IPython.display import Video

output_dir = 'output'
# 递归查找所有以 final_ 开头的 mp4 文件
final_videos = glob.glob(os.path.join(output_dir, '**', 'final_*.mp4'), recursive=True)

if final_videos:
    # 按照文件修改时间排序，展示最新生成的一个
    latest_video = max(final_videos, key=os.path.getmtime)
    print(f"🎬 展示最新生成的视频: {latest_video}")
    display(Video(latest_video, embed=True, width=600))
else:
    print("❌ 未找到生成的视频文件。")